In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import ast

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting


bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
notebook_url = 'localhost:8888'

In [108]:
# Read in data
TimeData = pd.read_csv(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetNew.csv"
)

# Prep dataset from data
TimeMelted = pd.melt(
    TimeData,
    id_vars=["Training", "ObjectsReinforced", "Reinforced_side", "Date", "Fly"],
    value_name="Values",
    value_vars=[
        "Visits Left Corner",
        "Durations Left Corner",
        "Visits Right Corner",
        "Durations Right Corner",
        "Visits Top Corner",
        "Durations Top Corner",
        "Visits Left Front",
        "Durations Left Front",
        "Visits Right Front",
        "Durations Right Front",
        "Visits Top Front",
        "Durations Top Front",
    ],
    var_name="Variable",
)

TimeMelted[["Values",]] = TimeMelted[
    [
        "Values",
    ]
].applymap(ast.literal_eval)
TimeMelted = TimeMelted.explode("Values")#.drop_duplicates()#.dropna()#.reset_index()
TimeMelted["EventIndex"] = TimeMelted.groupby(["Fly", "Variable"]).cumcount()
TimeMelted = TimeMelted.drop_duplicates().dropna()

conditions = [
    (TimeMelted["Variable"].str.contains("Durations")),
    (TimeMelted["Variable"].str.contains("Visits")),
]

values = ["Durations", "Visits"]
TimeMelted["Kind"] = np.select(conditions, values)

conditions = [
    (
        TimeMelted["Reinforced_side"].str.contains("Right")
        & (TimeMelted["Variable"].str.contains("Right"))
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Left")
        & (TimeMelted["Variable"].str.contains("Left"))
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Right")
        & (TimeMelted["Variable"].str.contains("Left"))
    ),
    (
        TimeMelted["Reinforced_side"].str.contains("Left")
        & (TimeMelted["Variable"].str.contains("Right"))
    ),
    (TimeMelted["Variable"].str.contains("Top")),
]

values = [
    "Rewarded Side",
    "Rewarded Side",
    "Punished Side",
    "Punished Side",
    "Empty Side",
]
TimeMelted["Condition"] = np.select(conditions, values)

TimeMelted = TimeMelted#.reset_index()

In [104]:
print(TimeData["Visits Left Corner"][1])
print(TimeData["Durations Left Corner"][1])

[452, 2689, 4476, 4538, 6189, 6324, 11393, 11760, 20790, 24333, 24453, 32411, 42102, 42900, 43139]
[1, 27, 32, 3, 37, 8, 19, 49, 32, 20, 46, 25, 54, 76, 88]


In [ ]:
print(TimeData["Visits Top Corner Corner"][1])
print(TimeData["Durations Left Corner"][1])

In [86]:
TimeMelted = TimeMelted.stack('Kind')

KeyError: 'Requested level (Kind) does not match index name (None)'

In [109]:
TimeMelted = (
    TimeMelted.reset_index()
    .pivot_table(
        index=["Condition","Fly","EventIndex","Training", "ObjectsReinforced", ],
        columns="Kind",
        values="Values",
    )
    .reset_index()
)

In [70]:

# Produce the subsets

ReinforcedVisit = TimeMelted[TimeMelted["Condition"] == "Rewarded Side"]

In [71]:

ReinforcedVisit.loc[
    ReinforcedVisit["Variable"].str.contains("Visits"), "Variable"
] = "Visits"
ReinforcedVisit.loc[
    ReinforcedVisit["Variable"].str.contains("Durations"), "Variable"
] = "Durations"

In [48]:
#ReinforcedVisit.unstack("Variable")

KeyError: 'Requested level (Variable) does not match index name (None)'

In [72]:

ReinforcedVisit = (
    ReinforcedVisit.reset_index()
    .pivot_table(
        index=["Fly","Training", "ObjectsReinforced", ],
        columns="Variable",
        values="Values",
    )
    .reset_index()
)

In [44]:

PunishedVisits = TimeMelted[TimeMelted["Condition"] == "Punished Side"]

PunishedVisits.loc[
    PunishedVisits["Variable"].str.contains("Visits"), "Variable"
] = "Visits"
PunishedVisits.loc[
    PunishedVisits["Variable"].str.contains("Durations"), "Variable"
] = "Durations"

PunishedVisits = (
    PunishedVisits.reset_index()
    .pivot_table(
        index=["Fly",
               "Training",
               "ObjectsReinforced"],
        columns="Variable",
        values="Values",
    )
    .reset_index()
)

EmptyVisits = TimeMelted[TimeMelted["Condition"] == "Empty Side"]

EmptyVisits.loc[EmptyVisits["Variable"].str.contains("Visits"), "Variable"] = "Visits"
EmptyVisits.loc[
    EmptyVisits["Variable"].str.contains("Durations"), "Variable"
] = "Durations"

EmptyVisits = (
    EmptyVisits.reset_index()
    .pivot_table(
        index=["Fly", "Training", "ObjectsReinforced"],
        columns="Variable",
        values="Values",
    )
    .reset_index()
)

In [4]:
# Options for x and y selectors

xy_options = list(
    ReinforcedVisit.columns[ReinforcedVisit.columns.isin(["Visits", "Durations"])]
)  #'ObjectsReinforced','Training', 'Fly'

# Define the selector widgets

x_selector = bokeh.models.Select(
    title="x",
    options=xy_options,
    value="Visits",
    width=200,
)

y_selector = bokeh.models.Select(
    title="y",
    options=xy_options,
    value="Durations",
    width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by",
    options=[
        "none",
        "ObjectsReinforced",
        "Training",
        "Fly",
    ],
    value="none",
    width=200,
)

In [ ]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisit[x_selector.value], y=ReinforcedVisit[y_selector.value], 
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisit)

# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color", 
                  #legend_group="color",
                 )

In [ ]:
from bokeh.palettes import Category10
import itertools

colors = itertools.cycle(Category10[10])

colors

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

In [ ]:
colors = []

for fly, c in zip(ReinforcedVisit["Fly"].unique(), color):
    colors.append(c)

colors
#colors = colorgen() for flies in ReinforcedVisit["Fly"]

In [ ]:
def gfmt_callback(attr, new, old):
    """Callback for updating plot of GMFT results."""
    # Update color column
    if colorby_selector.value == "none":
        source.data["color"] = ["#1f77b3"] * len(ReinforcedVisit)
        
    elif colorby_selector.value == "ObjectReinforced":
        source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisit["ObjectsReinforced"]
        ]
        
    elif colorby_selector.value == "Training":
        source.data["color"] = [
            "#1f77b3" if Training == "Trained" else "#ff7e0e"
            for Training in ReinforcedVisit["Training"]
        ]

    elif colorby_selector.value == "Fly":
        source.data["color"] = [
            
            colors for flies in ReinforcedVisit["Fly"].unique()
        ]

    # Update x-data and axis label
    source.data["x"] = ReinforcedVisit[x_selector.value]
    p.xaxis.axis_label = x_selector.value

    # Update x-data and axis label
    source.data["y"] = ReinforcedVisit[y_selector.value]
    p.yaxis.axis_label = y_selector.value


# Connect selectors to callback
colorby_selector.on_change("value", gfmt_callback)
x_selector.on_change("value", gfmt_callback)
y_selector.on_change("value", gfmt_callback)

# Build the layout
gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

In [ ]:
def gfmt_app(doc):
    doc.add_root(gfmt_layout)

bokeh.io.show(gfmt_app, notebook_url=notebook_url)

In [ ]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisit["Visits"], y=ReinforcedVisit["Durations"], 
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisit)

# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color", 
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [159]:
def gfmt_callback(attr, new, old):
    """Callback for updating plot of GMFT results."""
    # Update color column
    if colorby_selector.value == "none":
        source.data["color"] = ["#1f77b3"] * len(ReinforcedVisit)
        
    elif colorby_selector.value == "ObjectReinforced":
        source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisit["ObjectsReinforced"]
        ]
        
    elif colorby_selector.value == "Training":
        source.data["color"] = [
            "#1f77b3" if Training == "Trained" else "#ff7e0e"
            for Training in ReinforcedVisit["Training"]
        ]

    elif colorby_selector.value == "Fly":
        source.data["color"] = [
            
            colors for flies in ReinforcedVisit["Fly"].unique()
        ]

    # Update x-data and axis label
    source.data["x"] = ReinforcedVisit[x_selector.value]
    p.xaxis.axis_label = x_selector.value

    # Update x-data and axis label
    source.data["y"] = ReinforcedVisit[y_selector.value]
    p.yaxis.axis_label = y_selector.value


# Connect selectors to callback
colorby_selector.on_change("value", gfmt_callback)
x_selector.on_change("value", gfmt_callback)
y_selector.on_change("value", gfmt_callback)

# Build the layout
gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

In [161]:
def gfmt_app(doc):
    doc.add_root(gfmt_layout)

bokeh.io.show(gfmt_app, notebook_url=notebook_url)

In [32]:
# Column data source
source = bokeh.models.ColumnDataSource(
    dict(x=ReinforcedVisit["Visits"],
         y=ReinforcedVisit["Durations"],
         Fly=ReinforcedVisit['Fly'],
         Training=ReinforcedVisit['Training'],
         ObjectsReinforced=ReinforcedVisit['ObjectsReinforced']
         
        )
)

# Add a column for colors; for now, all bokeh's default blue
source.data["color"] = ["#1f77b3"] * len(ReinforcedVisit)

source.data["color"]

['#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3']

In [13]:
# Make the plot
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color",
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [14]:
from bokeh.palettes import Category10
import itertools

colors = itertools.cycle(Category10[10])

colors

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

In [20]:
Flycolors = []

for fly, c in zip(ReinforcedVisit["Fly"].unique(), color):
    colors.append(c)

Flycolors
#colors = colorgen() for flies in ReinforcedVisit["Fly"]

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [23]:
source.data["color"] = colors


In [24]:
source.data["color"]

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf',
 '#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [25]:
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color",
                  #legend_group="color",
                 )

bokeh.io.show(p)

In [26]:
source.data["color"] = [
            "#1f77b3" if objects == "Blue" else "#ff7e0e"
            for objects in ReinforcedVisit["ObjectsReinforced"]
        ]

source.data["color"]

['#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#ff7e0e',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3',
 '#1f77b3']

In [33]:
p = bokeh.plotting.figure(
    frame_height=500,
    frame_width=500,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
    tooltips=[('fly', '@{Fly}'),
              ('Training state', '@Training'),
              ('Object reinforced', '@ObjectsReinforced')
             ],
)

# Populate glyphs
circle = p.circle(source=source, x="x", y="y", color="color",
                  #legend_group="color",
                 )

bokeh.io.show(p)